# First Simulation for CausalMARL

## TODO
1. 你昨天design的那个呢.感觉那个DR基本都比IS要好？感觉主要比IS好好了。主要看看可以怎么样比IS好 或者 差不多。
5. 这个setting bias很大；ATE基本估不出来？
1. Tuning:
    1. Two penalties in Susan: their CV method
    2. [w_hidden, Learning_rate] of NN: CV
    4. Now: manually fix some values
    3. **Q-V CV**: hard to choose neighbour actions. we can do that? but very large?   


1. 其他参数没看出什么规律？
    1. 加不加 sd_R 好像没有统一的趋势。还是加一个吧
    1. sd_O 也没有看出什么趋势
    1. sd_D 也没有看出什么趋势。。

time_dependent
n_neigh

bias increased so much?


fix a setting and discuss the trend with lambda: etc
I can add some pattern to D and O. but the pattern needs to be shared between them.
why no trend with T? 
怎么估计QV? sample-spliting?
change default values

policy1 直接改掉吧

还是depends on the setting 的
why variance 差别这么小?
不能很像 不然会出问题？
CV for NN

还没有好的tuning

## DONE!
2. T=48*7=336的？就相当于一个星期，这样variance大一点
4. setting里e_{t,g}^R可以稍微调大一点，让sd大一些
6. 是baseline估的太差了？那theshold也不用这么大[可以10-11-12-13, 或者9-10-11-12, pi_0那个估的不行就不和pi_0比了]

In [ ]:
1. O follows poisson (larger variance?)
2. larger noise
sd_D = 10 #1
sd_R = 10 #1

要比non-spatial好很多？
M_in_R is True

thre_range = [80, 85, 90, 95, 100, 105, 110]
location 太多了？
看看规律吧

smaller bias. how?

present trend with sample size? 试一试？

找出最优的 (how?) 然后线性 组合？

n_neigh 慢慢琢磨? 

simple = True + small T + smaller sd
focus on pattern 0

同时改了还几个
如何把difference 弄大一些？

T: no trend
MSE
bias
spatial
consistency

target 换成其他几个random的 fixed 的? 

or just randomly  generate the target policy
要实际一些

## Target
1. value estimations are all good
2. difference estimations are all good
3. better than IS in 1, 2
4. better than single agent (in bias) in 1, 2
5. better than all neigh infomation (in variance) in 1, 2

## Tuning
1. Noise
    1. sd_O
    2. sd_R
    3. sd_D
1. D_t = (D_t + u_D) / 2
2. spatial
    1. w_O
    2. w_A
2. hyperparameters of NN and Q/V
2. Difference
    1. sd_u_O
    2. u_D = np.mean(u_O) - 2
3. scale?   
4. reward def
5. T
6. l
7. thre_range
8. pattern seed
9. p_behav
10. n_neigh (simple = False)


# Setting
### Model
0. Mis
    1. Size: grid, N = 5 * 5, T = 48 * 14
    2. at time t, based on $O_t$ and $D_t$ which are observations  during (t-1, t], we (randomly) choose action $A_t$ which will be implemented during (t, t + 1], and observe the reward $R_t$ which is defind as the mismatch during (t, t + 1].
1. State:
    1. Order: 
        1. $u^O_{l} \sim logN(2.5, .3)$  (mean = 12.5, std = 3)
        2. $O_{l,t} \sim u^O_{l} + e^O_{l,t}, e^O_{l,t} \sim (Poisson(1) - 1)$
    2. Driver: 
        1. Attraction of $l$: $Att_{l,t} = exp(w_A * A_{l,t}) + w_O * (\frac{O_{l,t}}{1 + D_{l,t}})$
        2. Flow:  $D_{l, t + 1} = \sum_{i \in N_l} (\frac{Att_{l,t}}{\sum_{j \in N_j}Att_{j,t}}D_{i,t} )$
        3. Computation: matrix multiplication
    4. Mismatch: $M_{t+1,g} = 0.5 * (1-\frac{|D_{t+1,g}-O_{t+1,g}|}{|1 + D_{t+1,g}+O_{t+1,g}|}) + 0.5 * M_{t,g}$
4. Reward: $R_{t,g} = M_{t + 1, g}min(D_{t + 1,g}, O_{t + 1,g}) + e^R_{t,g}$ 
3. Behav policy: $A_{t,g} \sim Bernoulli(0.5)$
5. Target policy:  $A_{t,g} = I_{\{u^O_l \ge 12\}}$

## Archive
2. $\vec{D}_{t+1} = \frac{1}{2}(\omega_t^{-1} (\mathbf{O}_t \times (\mathbf{A}_{t} + \mathbf{I}) \times \mathbf{Adj}  \times (\mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + 10) + \vec{\epsilon}^D_{t+1}$
3. $\vec{D}_{t+1} = \frac{1}{2}(\omega_t^{-1} (\mathbf{O}_t \times (\mathbf{A}_{t} + \mathbf{I}) \times \mathbf{Adj}  \times (\mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + 10 * (2 - sin(t/48 * 2\pi))) + \vec{\epsilon}^D_{t+1}$
2. $O_{t,g} \sim Poisson(10 * (2 - sin(t/48 * 2\pi)))$ [no heterogeneity] -> mean is 20?
2. $R_{i,t}=M_{i,t} * min(D_{i,t}, O_{i,t})+mean \{M_{j,t} * min(D_{j,t}, O_{j,t})\}_{j\in N(i)}+\epsilon_{i,t}$
1. $A_{t,g} = a_g, a_g \sim Bernoulli(0.5)$
2. $M_{t+1,g} = 1-\frac{|D_{t+1,g}-O_{t+1,g}|}{|D_{t+1,g}+O_{t+1,g}|}$
2. $O_{t,g} \sim Poisson(10)$
1. $\vec{D}_{t+1} = \omega_t^{-1} \mathbb{P}(((w_O * \mathbf{O}_t + \mathbf{I}) \times (w_A * \mathbf{A}_{t} +  \mathbf{I}) \times \mathbf{Adj}  \times (w_O * \mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + \vec{\epsilon}^D_{t+1})$
            1. $w_t$ is a normalization parameter, $\mathbf{A}_{t}$ and $\mathbf{O}_t$ are corresponding diagonal matries, and $\vec{\epsilon}^D_t \sim (Poisson(1) - 1)$
            2. operator $\mathbb{P}$ is defined as $\mathbb{P}(\vec{D}) = (max(int(D_1),1),\dots, max(int(D_N),1))^T$
            
1. + \epsilon^R_{t,g}, \epsilon^R_{t,g} \sim N(0,sd_R)$

### Metrics and competing methods
1. Ground truth: MC-based proxy
2. Metrics: bias, std and MSE (100 replicates)
3. Competing methods: 
    1. "IS": Importance Sampling (Lihong Li) + mean-field
    2. "Susan": Value funcions (Susan) + mean-field
    3. "DR_NS": DR without considerinig other agents
    2. DR w/o mean-field: $R_it$ with neigh states/actions (multi-dim actions)
8. Report results for one $\{u_{o,g}\}_g$ pattern with four potential target policies.

1. IS:
    1. single agent - bias
    2. spaatial - large variance. [看draft]
2. DR:
    1. better than IS
    
### Time cost

1. l = 5: 11mins for 2 reps and one setting/pattern